# Evaluating Model Performance for Predicting 30-Day Hospital Readmissions

Eric Jia, Scott Yamamoto

# Overview

This is the first of two notebooks and contains the code used to pre-process the MIMIC-3 data for model training. The second notebook uses the processed data from the end of this one in order to compare different models' performances.

Imports

In [1]:
!pip install --upgrade pip
!pip install pyarrow

In [1]:
import pandas as pd
import numpy as np

# Load CSV files with only necessary columns

We use the data from 6 different tables within MIMIC-3. To access these files, you must first register as a user and complete the mandatory CITI training and Data Use Agreement here: https://physionet.org/content/mimiciii/1.4/

In [48]:
labevents_df = pd.read_csv(
    "/Users/ericjia/Downloads/MIMIC-3 DAta/LABEVENTS.csv",
    usecols=["SUBJECT_ID", "HADM_ID", "ITEMID", "VALUENUM"]
)

chartevents_df = pd.read_csv(
    "/Users/ericjia/Downloads/MIMIC-3 DAta/CHARTEVENTS.csv",
    usecols=["HADM_ID", "ITEMID", "VALUENUM"]
)

outputevents_df = pd.read_csv(
    "/Users/ericjia/Downloads/MIMIC-3 DAta/OUTPUTEVENTS.csv",
    usecols=["HADM_ID", "ITEMID", "VALUE"]
)

patients_df = pd.read_csv(
    "/Users/ericjia/Downloads/MIMIC-3 DAta/PATIENTS.csv",
    usecols=["SUBJECT_ID", "DOB", "GENDER"]
)

admissions_df = pd.read_csv(
    "/Users/ericjia/Downloads/MIMIC-3 DAta/ADMISSIONS.csv",
    usecols=["HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME", "MARITAL_STATUS", "INSURANCE"]
)

icustays_df = pd.read_csv(
    "/Users/ericjia/Downloads/MIMIC-3 DAta/ICUSTAYS.csv",
    usecols=["HADM_ID", "FIRST_CAREUNIT", "LAST_CAREUNIT"]
)

# Convert datatypes for memory efficiency & merges

Convert the datatypes for memory efficiency. Additionally, convert times to datetime format to standardize time series data.

In [49]:
admissions_df["ADMITTIME"] = pd.to_datetime(admissions_df["ADMITTIME"])
admissions_df["DISCHTIME"] = pd.to_datetime(admissions_df["DISCHTIME"])
admissions_df["DEATHTIME"] = pd.to_datetime(admissions_df["DEATHTIME"])
patients_df["DOB"] = pd.to_datetime(patients_df["DOB"])

labevents_df["SUBJECT_ID"] = labevents_df["SUBJECT_ID"].astype("Int64")
labevents_df["HADM_ID"] = labevents_df["HADM_ID"].astype("Int64")
chartevents_df["HADM_ID"] = chartevents_df["HADM_ID"].astype("Int64")
outputevents_df["HADM_ID"] = outputevents_df["HADM_ID"].astype("Int64")
patients_df["SUBJECT_ID"] = patients_df["SUBJECT_ID"].astype("Int64")
admissions_df["HADM_ID"] = admissions_df["HADM_ID"].astype("Int64")
icustays_df["HADM_ID"] = icustays_df["HADM_ID"].astype("Int64")

# Define helper functions and aggregate the tables

In [50]:
def aggregate_lab(df, itemid, prefix):
    subset = df[df["ITEMID"] == itemid].copy()
    aggs = subset.groupby(["SUBJECT_ID", "HADM_ID"])["VALUENUM"].agg(["min","max","mean"])
    aggs.rename(columns={"min": f"{prefix}_MIN", "max": f"{prefix}_MAX", "mean": f"{prefix}_MEAN"}, inplace=True)
    return aggs.reset_index()

def aggregate_chart(df, itemids, prefix, valid_min=None, valid_max=None):
    subset = df[df["ITEMID"].isin(itemids)].copy()
    if valid_min is not None and valid_max is not None:
        subset = subset[subset["VALUENUM"].between(valid_min, valid_max)]
    aggs = subset.groupby("HADM_ID")["VALUENUM"].agg(["min","max","mean"])
    aggs.rename(columns={"min": f"{prefix}_MIN", "max": f"{prefix}_MAX", "mean": f"{prefix}_MEAN"}, inplace=True)
    return aggs.reset_index()

def aggregate_output(df, itemids):
    subset = df[df["ITEMID"].isin(itemids)].copy()
    aggs = subset.groupby("HADM_ID")["VALUE"].agg(["min","max","mean"])
    aggs.rename(columns={"min": "URINE_MIN", "max": "URINE_MAX", "mean": "URINE_MEAN"}, inplace=True)
    return aggs.reset_index()

In [51]:
# aggregate the data from LABEVENTS.csv

lab_urea = aggregate_lab(labevents_df, 51006, "UREA_N")
lab_platelets = aggregate_lab(labevents_df, 51265, "PLATELETS")
lab_mag = aggregate_lab(labevents_df, 50960, "MAGNESIUM")
lab_albumin = aggregate_lab(labevents_df, 50862, "ALBUMIN")
lab_calcium = aggregate_lab(labevents_df, 50893, "CALCIUM")

lab_aggs = (
    lab_urea
    .merge(lab_platelets, on=["SUBJECT_ID","HADM_ID"], how="outer")
    .merge(lab_mag, on=["SUBJECT_ID","HADM_ID"], how="outer")
    .merge(lab_albumin, on=["SUBJECT_ID","HADM_ID"], how="outer")
    .merge(lab_calcium, on=["SUBJECT_ID","HADM_ID"], how="outer")
)

lab_aggs.to_csv("/Users/ericjia/Downloads/lab_aggs.csv")

In [53]:
# aggregate the data from CHARTEVENTS.csv

resp_itemids = [615, 618, 220210, 224690]
resp_df = aggregate_chart(chartevents_df, resp_itemids, "RESP_RATE", 0, 70)

hr_itemids = [211, 220045]
hr_df = aggregate_chart(chartevents_df, hr_itemids, "HR", 0, 300)

glucose_itemids = [807, 811, 1529, 3745, 3744, 225664, 220621, 226537]
glucose_df = aggregate_chart(chartevents_df, glucose_itemids, "GLUCOSE", 0, 10000)

sysbp_itemids = [51, 442, 455, 6701, 220179, 220050]
sysbp_df = aggregate_chart(chartevents_df, sysbp_itemids, "SYSBP", 0, 400)

diasbp_itemids = [8368, 8440, 8441, 8555, 220180, 220051]
diasbp_df = aggregate_chart(chartevents_df, diasbp_itemids, "DIASBP", 0, 300)

# Merge all tables
chart_aggs = (
    resp_df
    .merge(hr_df, on="HADM_ID", how="outer")
    .merge(glucose_df, on="HADM_ID", how="outer")
    .merge(sysbp_df, on="HADM_ID", how="outer")
    .merge(diasbp_df, on="HADM_ID", how="outer")
)

chart_aggs.to_csv("/Users/ericjia/Downloads/chart_aggs.csv")

In [61]:
# aggregate the data from OUTPUTEVENTS.csv

output_agg = aggregate_output(outputevents_df, [40055, 226559])

# meerge all tables Everything (no SAPSII/SOFA)
merged_df = (
    lab_aggs
    .merge(output_agg, on="HADM_ID", how="inner")
    .merge(patients_df, on="SUBJECT_ID", how="inner")
    .merge(admissions_df, on="HADM_ID", how="inner")
    .merge(chart_aggs, on="HADM_ID", how="inner")
    .merge(icustays_df, on="HADM_ID", how="inner")
)

In [76]:
# select & rename columns

final_columns = [
    "SUBJECT_ID", "HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME",
    "FIRST_CAREUNIT", "LAST_CAREUNIT", "GENDER", "MARITAL_STATUS", "INSURANCE",
    "UREA_N_MIN", "UREA_N_MAX", "UREA_N_MEAN",
    "PLATELETS_MIN", "PLATELETS_MAX", "PLATELETS_MEAN",
    "MAGNESIUM_MIN", "MAGNESIUM_MAX", "MAGNESIUM_MEAN",
    "ALBUMIN_MIN", "ALBUMIN_MAX", "ALBUMIN_MEAN",
    "CALCIUM_MIN", "CALCIUM_MAX", "CALCIUM_MEAN",
    "RESP_RATE_MIN", "RESP_RATE_MAX", "RESP_RATE_MEAN",
    "HR_MIN", "HR_MAX", "HR_MEAN",
    "SYSBP_MIN", "SYSBP_MAX", "SYSBP_MEAN",
    "DIASBP_MIN", "DIASBP_MAX", "DIASBP_MEAN",
    "GLUCOSE_MIN", "GLUCOSE_MAX", "GLUCOSE_MEAN",
    "URINE_MIN", "URINE_MEAN", "URINE_MAX"
]

# keep only existing columns
existing_cols = [c for c in final_columns if c in merged_df.columns]
final_df = merged_df[existing_cols].sort_values(["SUBJECT_ID","ADMITTIME"])
print("Final shape:", final_df.shape)
print(final_df.head())

Final shape: (44622, 43)
   SUBJECT_ID  HADM_ID           ADMITTIME           DISCHTIME  \
0           3   145834 2101-10-20 19:08:00 2101-10-31 13:58:00   
1           6   107064 2175-05-30 07:15:00 2175-06-15 16:00:00   
2           9   150750 2149-11-09 13:06:00 2149-11-14 10:15:00   
3          12   112213 2104-08-07 10:15:00 2104-08-20 02:57:00   
4          13   143045 2167-01-08 18:43:00 2167-01-15 15:15:00   

            DEATHTIME FIRST_CAREUNIT LAST_CAREUNIT GENDER MARITAL_STATUS  \
0                 NaT           MICU          MICU      M        MARRIED   
1                 NaT           SICU          SICU      F        MARRIED   
2 2149-11-14 10:15:00           MICU          MICU      M            NaN   
3 2104-08-20 02:57:00           SICU          SICU      M        MARRIED   
4                 NaT            CCU          CSRU      F            NaN   

  INSURANCE  ...  SYSBP_MEAN  DIASBP_MIN  DIASBP_MAX  DIASBP_MEAN  \
0  Medicare  ...  116.767045         0.0       103.0

# Determine whether each pt is re-admitted within 30 days

This section mainly involves determining whether each patient was readmitted within 30 days.

Basic exploratory data analysis

In [71]:
print(final_df[['UREA_N_MIN', 'UREA_N_MEAN', 'UREA_N_MAX']].describe().loc[['mean', 'std']], '\n')
print(final_df[['MAGNESIUM_MAX', 'ALBUMIN_MIN', 'CALCIUM_MIN']].describe().loc[['mean', 'std']], '\n')
print(final_df[['RESP_RATE_MIN', 'RESP_RATE_MEAN', 'RESP_RATE_MAX']].describe().loc[['mean', 'std']], '\n')
print(final_df[['GLUCOSE_MIN', 'GLUCOSE_MEAN', 'GLUCOSE_MAX']].describe().loc[['mean', 'std']], '\n')
print(final_df[['HR_MIN', 'HR_MEAN', 'HR_MAX']].describe().loc[['mean', 'std']], '\n')
print(final_df[['SYSBP_MIN', 'SYSBP_MEAN', 'SYSBP_MAX']].describe().loc[['mean', 'std']], '\n')
print(final_df[['DIASBP_MIN', 'DIASBP_MEAN', 'DIASBP_MAX']].describe().loc[['mean', 'std']], '\n')
print(final_df[['URINE_MIN', 'URINE_MEAN', 'URINE_MAX']].describe().loc[['mean', 'std']], '\n')

      UREA_N_MIN  UREA_N_MEAN  UREA_N_MAX
mean   16.654032    25.931473   38.008854
std    13.610900    18.676178   29.019086 

      MAGNESIUM_MAX  ALBUMIN_MIN  CALCIUM_MIN
mean       2.437725     2.933336     7.776259
std        0.924376     0.753183     0.841790 

      RESP_RATE_MIN  RESP_RATE_MEAN  RESP_RATE_MAX
mean       9.123511       19.150351      31.668871
std        5.030956        3.583524       8.711681 

      GLUCOSE_MIN  GLUCOSE_MEAN  GLUCOSE_MAX
mean    90.588721    137.149771   217.015159
std     31.211948     35.664115   127.363413 

         HR_MIN    HR_MEAN      HR_MAX
mean  61.961123  85.253221  114.401428
std   18.930170  13.492709   24.599291 

      SYSBP_MIN  SYSBP_MEAN   SYSBP_MAX
mean  75.182391  120.359561  162.839337
std   32.549534   15.938392   28.158213 

      DIASBP_MIN  DIASBP_MEAN  DIASBP_MAX
mean   34.506338    60.406628   96.555374
std    16.139765     9.959304   26.208114 

      URINE_MIN  URINE_MEAN     URINE_MAX
mean  29.996947  131.179304  

In [77]:
final_df["ADMITTIME"] = pd.to_datetime(final_df["ADMITTIME"])
final_df["DISCHTIME"] = pd.to_datetime(final_df["DISCHTIME"])

# sort by SUBJECT_ID and ADMITTIME to ensure chronological order
final_df = final_df.sort_values(by=["SUBJECT_ID", "ADMITTIME"])

# initialize new columns
final_df["READMIT_DT"] = 0.0
final_df["NEXT_READMIT_DT"] = 0.0
final_df["READMIT_LAST_CAREUNIT"] = None

# calculate time delta for readmissions
for idx in range(1, final_df.shape[0]):
    if final_df.iloc[idx]["SUBJECT_ID"] == final_df.iloc[idx - 1]["SUBJECT_ID"]:
        prev_disch = final_df.iloc[idx - 1]["DISCHTIME"]
        curr_adm = final_df.iloc[idx]["ADMITTIME"]

        # convert to hours
        dt = (curr_adm - prev_disch).total_seconds() / 3600  
        dt_hrs_calc = round(dt, 2)

        # assign values
        final_df.at[idx, "READMIT_DT"] = dt_hrs_calc
        final_df.at[idx - 1, "NEXT_READMIT_DT"] = dt_hrs_calc
        final_df.at[idx, "READMIT_LAST_CAREUNIT"] = final_df.iloc[idx - 1]["LAST_CAREUNIT"]

final_df.shape

(44622, 46)

In [78]:
final_df = final_df.drop(columns=['URINE_MIN', 'URINE_MEAN', 'URINE_MAX'], errors='ignore')

# remove rows where READMIT_DT is negative
final_df = final_df[final_df["READMIT_DT"] >= 0]

# remove cases where the patient died during their stay
final_df = final_df[final_df["DEATHTIME"].isna()]

# drop the DEATHTIME column to prevent unnecessary data loss when dropping NaNs
final_df = final_df.drop(columns=['DEATHTIME'], errors='ignore')

# drop rows with NaNs in all columns except the last column
final_df = final_df.dropna(subset=final_df.columns[:-1]).reset_index(drop=True)

In [79]:
final_df.shape

(20948, 42)

In [80]:
# define the threshold in hours (30 days)
threshold = 30 * 24  

# initialize FUTURE_READMIT column
final_df["FUTURE_READMIT"] = np.where(
    final_df["NEXT_READMIT_DT"] == 0.0, "No",
    np.where(final_df["NEXT_READMIT_DT"] <= threshold, "Yes", "No")
)

# display value counts and proportions
print("Value counts:")
print(final_df["FUTURE_READMIT"].value_counts(), "\n")

print("Value proportions:")
print(final_df["FUTURE_READMIT"].value_counts(normalize=True))

Value counts:
FUTURE_READMIT
No     18184
Yes     2764
Name: count, dtype: int64 

Value proportions:
FUTURE_READMIT
No     0.868054
Yes    0.131946
Name: proportion, dtype: float64


In [6]:
# aggregate the data from NOTEEVENTS.csv
noteevents_df = pd.read_csv('/Users/ericjia/Downloads/MIMIC-3 DAta/NOTEEVENTS.csv')

noteevents_unique = noteevents_df.groupby("HADM_ID")["TEXT"].first().reset_index()

# merge with final_df on HADM_ID to get the discharge summaries
final_df = final_df.merge(noteevents_unique, on="HADM_ID", how="left")
final_df.shape

In [6]:
final_df.to_csv("/Users/ericjia/Downloads/543 project/full_final_df.csv")